<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/Other/Analysis/RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine
import numpy as np
from google.colab import userdata
SQLALCHEMY_SILENCE_UBER_WARNING=1

In [ ]:
# подключение к БД
engine = create_engine(userdata.get('simulative'))

In [16]:
# для удобства напишем функцию для загрузки результатов sql запросов
def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [17]:
# запросим предагреггированные данные
sql = """
SELECT
  card as client_id,
  Max((SELECT max(datetime) :: date FROM bonuscheques) - datetime :: date) as days_passed,
  count(*) as cnt,
  sum(summ_with_disc) as summ
FROM
  bonuscheques
GROUP BY
  card
"""
data = select(sql)

In [21]:
# посмотрим, как выглядят данные
data.head()

,client_id,days_passed,cnt,summ
0,ece474c7-8a08-4853-8e6c-bc2d7043df7d,328,25,11577
1,2000200229544,234,2,1657
2,564a56ba-d8ae-43e4-ab1a-702198cb0c4c,317,14,12111
3,2000200171775,236,2,1033
4,2000200229094,165,2,1235


In [22]:
data.describe()

,days_passed,cnt,summ
count,9394.000000,9394.00000,9394.000000
mean,202.197147,4.09687,3416.820098
std,98.744462,6.43155,6388.046844
min,0.000000,1.00000,21.000000
25%,123.000000,1.00000,731.250000
50%,223.000000,2.00000,1586.000000
75%,290.000000,5.00000,3731.750000
max,332.000000,217.00000,162687.000000


Видим аномальный максимум в признаке cnt (Кто-то покупает почти каждый день?)

In [30]:
data["log_summ"] = data.summ.apply(np.log)

In [35]:
data.head()

,client_id,days_passed,cnt,summ,log_summ
0,ece474c7-8a08-4853-8e6c-bc2d7043df7d,328,25,11577,9.356776
1,2000200229544,234,2,1657,7.412764
2,564a56ba-d8ae-43e4-ab1a-702198cb0c4c,317,14,12111,9.401869
3,2000200171775,236,2,1033,6.940222
4,2000200229094,165,2,1235,7.118826


In [36]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler